In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize, sent_tokenize
import operator
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np


In [89]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xizheng/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [356]:
with open("WSD/wsd_data.xml") as f:
    unformat_data = f.readlines()
    f.close()

In [381]:
def defalt_wsd_data():  #Since I always change the element in wsd_data, I defalt it when building every feature
    wsd_data = {}
    for i in range(len(unformat_data)):
        if unformat_data[i].startswith('<welt') == True:
            wsd_data[unformat_data[i][:-1]] = []
        else:
            if unformat_data[i] != '\n':
                     wsd_data[list(wsd_data)[len(wsd_data)-1]].append(unformat_data[i][:-1]) 
                #Make wsd_data becomes a dictionary, each word contains their own instance
    return wsd_data

In [358]:
def simplify_feature(dic,wsd_data): # Same as above
    #Simplify_wsd_BOWfeature = {}

    for i in wsd_data.keys():    
        big_list = []
        big_list_simplify = []
        for j in wsd_data[i]:
            if j == '</welt>':
                wsd_data[i] = big_list
                dic[i] = big_list_simplify
            
                break
            if j.startswith('<instance'):
                temp = [j]
                temp_simplify = []
            else:
                if j.startswith('<ans'):
                    temp_simplify.append(j.split(' ')[2][:-2])
                if j.startswith('<') == False:
                    temp_simplify.append(j)
                if j.startswith('</instance') == False:
                    temp.append(j)
                else:
                    temp.append(j)
                    big_list.append(temp)
                    big_list_simplify.append(temp_simplify)
    return dic
#Improve wsd_data dictionary, each instance belongs to their own list, and all the instances that 
#contains one welt belongs a big list, which is the value of the key welt

In [359]:
def tokenit(text):
    text_token = wordpunct_tokenize(text)
    return text_token

In [360]:
def vocabulary(wsd_dic):
    vocabulary_dic = {}
    for i in wsd_dic.keys():
        word_count = {}
        updated_word_count = {}
        for j in wsd_dic[i]:
            contex = j[-1].replace('<head>', '').replace('</head>', '')
            
            tokened_text = tokenit(contex)
            for x in tokened_text:
                if x in word_count:
                    word_count[x] += 1
                else:
                    word_count[x] = 1
        for k in word_count:
            if word_count[k] >= 20: # I choose words that appears more than 20 times in all context for one head word
                                    # to be in my vocabulary for that word
                updated_word_count[k] = word_count[k]
        vocabulary_dic[i] = updated_word_count
    return vocabulary_dic

def bag_of_words_feature(text_token, one_word_vocabulary_dic):
    
    vector=[]
    for i in one_word_vocabulary_dic.keys():
        if i in text_token:
            vector.append(1)
        else:
            vector.append(0)
    
    return vector

In [361]:
Simplify_wsd_BOWfeature = simplify_feature({}, defalt_wsd_data())

In [362]:
Head_Word_Vocabulary = vocabulary(Simplify_wsd_BOWfeature)

In [363]:
for welt, attribute in Simplify_wsd_BOWfeature.items(): 
    
    for i in attribute:
        
        context = i[-1].replace('<head>', '').replace('</head>', '') # abandon head tag
        tokened_context = tokenit(context)
        bow_vector = bag_of_words_feature(tokened_context,Head_Word_Vocabulary[welt])
        Feature_Selection_X
        i.append(bow_vector)

In [391]:
with open('2.2.1_Bag_of_Words.txt', 'w') as f:
    for i, j in Simplify_wsd_BOWfeature.items():
        f.writelines('{' +i+ ': \n')
        f.writelines(str(j) + '}\n')
    f.close()


In [392]:
with open('2.2.1_Bag_of_Words_vocabulary.txt', 'w') as f:
    for i, j in Head_Word_Vocabulary.items():
        f.writelines('{' +i+ ': \n')
        f.writelines(str(j) + '}\n')
    f.close()


### collocation feature

In [394]:
Simplify_wsd_CollocationFeature = simplify_feature({}, defalt_wsd_data())

In [395]:
for welt, attribute in Simplify_wsd_CollocationFeature.items(): 
    for i in attribute:
        context = i[-1] # abandon head tag
        tokened_context =context.split(' ')
        off_empty = []
        for m in tokened_context:
            if m != '':
                off_empty.append(m)
        #print(off_empty)
        
        for k in off_empty:
            if '<head>' in k:
                ind = off_empty.index(k)
                Wi_2 = off_empty[ind-2]
                Wi_1 = off_empty[ind-1]
                Wi1 = off_empty[ind+1]
                Wi2 = off_empty[ind+2]
                #print([Wi_2, Wi_1, Wi1, Wi2])
                POS_tag = nltk.pos_tag([Wi_2, Wi_1, Wi1, Wi2])
        
        i.append([POS_tag[0][0],POS_tag[0][1],POS_tag[1][0],POS_tag[1][1],POS_tag[2][0],POS_tag[2][1]
                  ,POS_tag[3][0],POS_tag[3][1], Wi_1+'_' + Wi_2, Wi1+'_'+Wi2])

In [396]:
with open('2.2.1_Collocation.txt', 'w') as f:
    for i, j in Simplify_wsd_CollocationFeature.items():
        f.writelines('{' +i+ ': \n')
        f.writelines(str(j) + '}\n')
    f.close()


### My new feature

In [440]:
def head_word_format(wsd_dic):
    format_dic = {}
    for i in wsd_dic.keys():
        format_set = set()
        
        for j in wsd_dic[i]:
            
            context = j[-1] # abandon head tag
            tokened_context =context.split(' ')
            
            for x in tokened_context:
                
                if '<head>' in x:
                    format_set.add(x)
        format_list = list(format_set)
        format_dic[i] = format_list
    return format_dic

In [441]:
Simplify_wsd_NewFeature = simplify_feature({}, defalt_wsd_data())

In [442]:
Format_vocabulary = head_word_format(Simplify_wsd_NewFeature)

In [443]:
for welt, attribute in Simplify_wsd_NewFeature.items(): 
    for i in attribute:
        context = i[-1] # abandon head tag
        tokened_context =context.split(' ')
        format_vector = []
        
        for k in tokened_context:
            if '<head>' in k:
                for j in Format_vocabulary[welt]:
                    if k == j:
                        format_vector.append(1)
                    else:
                        format_vector.append(0)
                break
        i.append(format_vector)

In [446]:
with open('2.2.1_MyNewFeature.txt', 'w') as f:
    for i, j in Simplify_wsd_NewFeature.items():
        f.writelines('{' +i+ ': \n')
        f.writelines(str(j) + '}\n')
    f.close()

In [445]:
with open('2.2.1_MyNewFeatureVocabulary.txt', 'w') as f:
    for i, j in Format_vocabulary.items():
        f.writelines('{' +i+ ': \n')
        f.writelines(str(j) + '}\n')
    f.close()

### Feature Selection

##### Before feature selections, I need to transform collocational feature to numbers instead of word strings and tag strings because I can't use strings to run through SelectKBest function. I decide to use the count of each collocation features.

In [414]:
Updated_Simplify_wsd_CollocationFeature = {}
collo_vocabulary = {}
for i in Simplify_wsd_CollocationFeature.keys():
    for j in Simplify_wsd_CollocationFeature[i]:
        collo_f = j[-1]
        for k in collo_f:
            if k in collo_vocabulary:
                collo_vocabulary[k] +=1
            else:
                collo_vocabulary[k] = 1

In [416]:
for x in Simplify_wsd_CollocationFeature.keys():
    collocation_numbers = []
    for j in Simplify_wsd_CollocationFeature[x]:
        collo_f = j[-1]
        empty = []
        for k in collo_f:
            empty.append(collo_vocabulary[k])
        collocation_numbers.append(empty)
    Updated_Simplify_wsd_CollocationFeature[x] = collocation_numbers

In [452]:
# Append all three features to one vector
Feature_Selection = {}
for i in Simplify_wsd_BOWfeature.keys():
    senseid = []
    senseid_corresponding_feature = []
    
    for j in range(len(Simplify_wsd_BOWfeature[i])):
        
        for k in range(len(Simplify_wsd_CollocationFeature[i][j])):
            
            if 'senseid=' in Simplify_wsd_CollocationFeature[i][j][k]:
                
                senseid.append(Simplify_wsd_CollocationFeature[i][j][k])
                
                Bow_f = Simplify_wsd_BOWfeature[i][j][-1]
                Collection_f = Updated_Simplify_wsd_CollocationFeature[i][j]
                New_f = Simplify_wsd_NewFeature[i][j][-1]
                
                senseid_corresponding_feature.append(Bow_f + Collection_f + New_f)
    Feature_Selection[i] = [senseid_corresponding_feature, np.asarray(senseid)]

In [457]:
#Run through SelectKBest, using get_support function to return the index of features that are selected.
feature_selection_dic = {}
for i in Feature_Selection.keys():
    X, y = Feature_Selection[i][0],Feature_Selection[i][1]
    selector = SelectKBest(chi2, k=10)
    selector.fit(X, y)

    X_new = selector.transform(X)

    feature_selection_dic[i] = selector.get_support(indices=True)

In [461]:
with open('2.2.2_FeatureSelection.txt', 'w') as f:
    for i, j in feature_selection_dic.items():
        f.writelines(i+ ': '+ str(j) + '\n')
    f.close()